In [41]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [42]:
filepath = tf.keras.utils.get_file('shakespeare.txt', './data/shakespeare.txt')

In [43]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [44]:
text = text[100000:1000000]

In [45]:
characters = sorted(set(text))

In [46]:
char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [97]:
SEQ_LENGTH = 40
STEP_SIZE = 3

In [98]:
sentences = []
next_characters = []

In [99]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
  sentences.append(text[i: i + SEQ_LENGTH])
  next_characters.append(text[i + SEQ_LENGTH])

In [100]:
X = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool_)
Y = np.zeros((len(sentences), len(characters)), dtype=np.bool_)

In [101]:
for i, sentence in enumerate(sentences):
  for t, character in enumerate(sentence):
    X[i, t, char_to_index[character]] = 1
  Y[i, char_to_index[next_characters[i]]] = 1

In [102]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
csv_logger = tf.keras.callbacks.CSVLogger('./training_log/log.csv', separator=',', append=True)

model.fit(X, Y, batch_size=256, epochs=10, callbacks=[csv_logger])

Epoch 1/10
1172/1172 [==============================] - 125s 103ms/step - loss: 1.9319
Epoch 2/10
1172/1172 [==============================] - 116s 99ms/step - loss: 1.5867
Epoch 3/10
1172/1172 [==============================] - 117s 100ms/step - loss: 1.5050
Epoch 4/10
1172/1172 [==============================] - 116s 99ms/step - loss: 1.4602
Epoch 5/10
1172/1172 [==============================] - 116s 99ms/step - loss: 1.4316
Epoch 6/10
1172/1172 [==============================] - 115s 99ms/step - loss: 1.4115
Epoch 7/10
1172/1172 [==============================] - 95s 81ms/step - loss: 1.3948
Epoch 8/10
1172/1172 [==============================] - 75s 64ms/step - loss: 1.3821
Epoch 9/10
1172/1172 [==============================] - 74s 63ms/step - loss: 1.3727
Epoch 10/10
1172/1172 [==============================] - 74s 63ms/step - loss: 1.3631


In [103]:
model.save('textgenerator.model')

INFO:tensorflow:Assets written to: textgenerator.model\assets


INFO:tensorflow:Assets written to: textgenerator.model\assets


In [87]:
model = tf.keras.models.load_model('textgenerator.model')

In [88]:
def sample(preds, temperature=1.0):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds)/temperature
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [89]:
def generate_text(length, temperature):
  start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
  generated = ''
  sentence = text[start_index: start_index + SEQ_LENGTH]
  generated += sentence
  for i in range(length):
    X = np.zeros((1, SEQ_LENGTH, len(characters)))
    for t, character in enumerate(sentence):
      X[0, t, char_to_index[character]] = 1
    
    predictions = model.predict(X, verbose=0)[0]
    next_index = sample(predictions, temperature)
    next_character = index_to_char[next_index]

    generated += next_character
    sentence = sentence[1:] + next_character
  
  return generated

In [90]:
print('-----------------------------0.2-------------------------------')
print(generate_text(300, 0.2))
print('-----------------------------0.4-------------------------------')
print(generate_text(300, 0.4))
print('-----------------------------0.6-------------------------------')
print(generate_text(300, 0.6))
print('-----------------------------0.8-------------------------------')
print(generate_text(300, 0.8))
print('------------------------------1--------------------------------')
print(generate_text(300, 1))

-----------------------------0.2-------------------------------
e sound that tells what hour it is
are clamorous grows with the world and the world,
they shall be a manners the way to my son,
the lord of the world be a man, and there is not
the and the winds the world and still the great all
and the that to the strind the sense, the to thee
to the mother and the waster and the state
of the world of your heart o
-----------------------------0.4-------------------------------
jove to humble him to her hand.
when with his knee in men with the sense,
the happy so man to be with his palsed
that is the greater of my lord and be the grace.

second servingment:
i princery, not with me the most grace.

second murdere:
there is the wind the way to the sweet as i am
nor the wind man to good pardon her heavens
that this way of th
-----------------------------0.6-------------------------------
 like to be loud weather;
besides, this place is from of his heads bethe
good clost the thought and in the